In [3]:
# Análise Genérica de Dados de Criminalidade
# ------------------------------------------

# Importações
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns

from openpyxl import load_workbook
from IPython.display import display
from fpdf import FPDF

# Configurações de exibição
plt.style.use('ggplot')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)


In [2]:
# Função: Carregamento de dados xlsx
caminho_pasta = '../../dados_tratados/PNAD_2009'

def carregar_dados_xlsx(caminho):
    df = pd.read_excel(caminho, sheet_name=None)

    return df

dados = {
    "agressao": carregar_dados_xlsx(os.path.join(caminho_pasta, 'agressao.xlsx')),
    "furto": carregar_dados_xlsx(os.path.join(caminho_pasta, 'furto.xlsx')),
    "roubo": carregar_dados_xlsx(os.path.join(caminho_pasta, 'roubo.xlsx')),
    "roubofurto": carregar_dados_xlsx(os.path.join(caminho_pasta, 'roubofurto.xlsx')),
    "seguranca": carregar_dados_xlsx(os.path.join(caminho_pasta, 'seguranca.xlsx')),
    "tentativa": carregar_dados_xlsx(os.path.join(caminho_pasta, 'tentativa.xlsx'))
}

In [ ]:
# ===================================================================
# CÉLULA 2: GERAÇÃO DO RELATÓRIO PDF FINAL
# ===================================================================

print("Iniciando a montagem do relatório em PDF...")

# Classe PDF personalizada com funções de ajuda
class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        data_hoje = pd.to_datetime('today').strftime('%d de %B de %Y')
        self.cell(0, 10, f'Relatório PNAD 2009 - {data_hoje}', 0, 1, 'C')
        self.ln(5)
    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Página {self.page_no()}', 0, 0, 'C')
    def chapter_title(self, title):
        self.set_font('Arial', 'B', 14)
        self.cell(0, 10, title, 0, 1, 'L')
        self.ln(5)
    def output_df_to_pdf(self, df):
        # Header
        self.set_font('Arial', 'B', 10)
        col_width = self.w / (len(df.columns) + 1.5)
        for col in df.columns:
            self.cell(col_width, 10, str(col).replace('_', ' ').title(), 1, 0, 'C')
        self.ln()
        # Body
        self.set_font('Arial', '', 10)
        for index, row in df.iterrows():
            for col in df.columns:
                value = row[col]
                if isinstance(value, float):
                    value = f'{value:.2f}'
                self.cell(col_width, 10, str(value), 1, 0, 'C')
            self.ln()
        self.ln(5)

# Iniciar PDF
pdf = PDF()
pdf.add_page()
pdf.set_font('Arial', 'B', 16)
pdf.cell(0, 10, 'Análise Combinada de Percepção de Segurança', 0, 1, 'C')
pdf.ln(10)

# Página 1: Risco por Sexo
pdf.chapter_title('1. Análise da Percepção de Risco por Sexo')
pdf.output_df_to_pdf(df_risco_sexo.set_index('sexo'))
pdf.image(caminho_grafico_sexo, w=170)

# Página 2: Risco por Bairro
pdf.add_page()
pdf.chapter_title('2. Análise da Percepção de Risco por Bairro')
pdf.output_df_to_pdf(df_risco_bairro.set_index('estrato_bairro'))
pdf.image(caminho_grafico_bairro, w=170)

# Página 3: Risco por Faixa de Idade
pdf.add_page()
pdf.chapter_title('3. Análise da Percepção de Risco por Faixa de Idade')
pdf.output_df_to_pdf(df_risco_idade.set_index('faixa_idade'))
pdf.image(caminho_grafico_idade, w=170)

# Página 4: Matriz de Correlação
pdf.add_page()
pdf.chapter_title('4. Matriz Numérica de Correlação entre Riscos')
pdf.set_font('Arial', '', 11)
pdf.multi_cell(0, 7, "A tabela e o mapa de calor abaixo mostram a correlação entre a percepção dos diferentes tipos de risco. Valores próximos de 1 indicam uma forte correlação positiva, significando que quem teme um tipo de crime também tende a temer o outro.")
pdf.ln(5)
pdf.image(caminho_grafico_correlacao, w=120, x=pdf.w / 2 - 60)

# Página 5: Conclusões
pdf.add_page()
pdf.chapter_title('5. Resultados e Conclusões')
texto_conclusao = (
    "A análise dos dados de percepção de segurança revela insights importantes sobre como diferentes "
    "grupos demográficos e sociais experienciam o medo em relação à criminalidade. A correlação positiva "
    "entre os tipos de risco sugere que a sensação de insegurança é um sentimento generalizado, não focado em um único tipo de crime.\n\n"
    "Observou-se que a percepção de risco não é homogênea, apresentando variações significativas "
    "quando segmentada por sexo, faixa etária e estrato do bairro de residência. "
    "Por exemplo, a análise por sexo indicou [descreva aqui a principal diferença, ex: que o público feminino percebe maior risco de sequestro].\n\n"
    "Estes resultados sugerem a necessidade de políticas de segurança pública que sejam sensíveis a "
    "essas diferentes percepções, focando esforços em localidades e grupos que demonstram maior "
    "sensação de insegurança."
)
pdf.set_font('Arial', '', 12)
pdf.multi_cell(0, 7, texto_conclusao.encode('latin-1', 'replace').decode('latin-1'))


# --- Salvamento e Limpeza ---
pasta_analises = os.path.join('..', '..', 'analises')
if not os.path.exists(pasta_analises):
    os.makedirs(pasta_analises)
caminho_arquivo_pdf = os.path.join(pasta_analises, "relatorio_completo_percepcao_social.pdf")

try:
    pdf.output(caminho_arquivo_pdf)
    caminho_absoluto = os.path.abspath(caminho_arquivo_pdf)
    print(f"\nPDF gerado com sucesso!")
    print(f"O arquivo foi salvo em: {caminho_absoluto}")
except Exception as e:
    print(f"\nOcorreu um erro ao gerar o PDF: {e}")
finally:
    for caminho in nomes_graficos:
        os.remove(caminho)
    os.rmdir(pasta_temp)
    print("\nArquivos temporários removidos.")

Iniciando a montagem do relatório em PDF...

PDF gerado com sucesso!
O arquivo foi salvo em: c:\Users\pedro\OneDrive\Documentos\GitHub\Repositorio_pesquisas\analises_vitimizacao\analises\relatorio_completo_percepcao_social.pdf

Arquivos temporários removidos.
